In [13]:
import numpy as np
import pandas as pd
import sklearn.metrics as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


In [14]:
ds = pd.read_csv(
    r"E:\SmartInternz_Pynb\Assignment_7\A7_RNN\heart_failure_clinical_records_dataset.csv")
ds


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [15]:
x = ds.iloc[:, :-1].values
y = ds.iloc[:, 12].values


In [16]:
x

array([[7.500e+01, 0.000e+00, 5.820e+02, ..., 1.000e+00, 0.000e+00,
        4.000e+00],
       [5.500e+01, 0.000e+00, 7.861e+03, ..., 1.000e+00, 0.000e+00,
        6.000e+00],
       [6.500e+01, 0.000e+00, 1.460e+02, ..., 1.000e+00, 1.000e+00,
        7.000e+00],
       ...,
       [4.500e+01, 0.000e+00, 2.060e+03, ..., 0.000e+00, 0.000e+00,
        2.780e+02],
       [4.500e+01, 0.000e+00, 2.413e+03, ..., 1.000e+00, 1.000e+00,
        2.800e+02],
       [5.000e+01, 0.000e+00, 1.960e+02, ..., 1.000e+00, 1.000e+00,
        2.850e+02]])

In [17]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtyp

In [18]:
ct = ColumnTransformer([("one", OneHotEncoder(), [])], remainder="passthrough")
x = ct.fit_transform(x)
x


array([[7.500e+01, 0.000e+00, 5.820e+02, ..., 1.000e+00, 0.000e+00,
        4.000e+00],
       [5.500e+01, 0.000e+00, 7.861e+03, ..., 1.000e+00, 0.000e+00,
        6.000e+00],
       [6.500e+01, 0.000e+00, 1.460e+02, ..., 1.000e+00, 1.000e+00,
        7.000e+00],
       ...,
       [4.500e+01, 0.000e+00, 2.060e+03, ..., 0.000e+00, 0.000e+00,
        2.780e+02],
       [4.500e+01, 0.000e+00, 2.413e+03, ..., 1.000e+00, 1.000e+00,
        2.800e+02],
       [5.000e+01, 0.000e+00, 1.960e+02, ..., 1.000e+00, 1.000e+00,
        2.850e+02]])

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)


In [20]:
x_train[0:5]

array([[8.10e+01, 0.00e+00, 4.54e+03, 0.00e+00, 3.50e+01, 0.00e+00,
        2.31e+05, 1.18e+00, 1.37e+02, 1.00e+00, 1.00e+00, 1.07e+02],
       [5.00e+01, 0.00e+00, 1.85e+02, 0.00e+00, 3.00e+01, 0.00e+00,
        2.66e+05, 7.00e-01, 1.41e+02, 1.00e+00, 1.00e+00, 1.12e+02],
       [4.50e+01, 0.00e+00, 5.82e+02, 0.00e+00, 3.50e+01, 0.00e+00,
        3.85e+05, 1.00e+00, 1.45e+02, 1.00e+00, 0.00e+00, 6.10e+01],
       [5.20e+01, 0.00e+00, 1.90e+02, 1.00e+00, 3.80e+01, 0.00e+00,
        3.82e+05, 1.00e+00, 1.40e+02, 1.00e+00, 1.00e+00, 2.58e+02],
       [5.00e+01, 1.00e+00, 5.40e+01, 0.00e+00, 4.00e+01, 0.00e+00,
        2.79e+05, 8.00e-01, 1.41e+02, 1.00e+00, 0.00e+00, 2.50e+02]])

In [21]:
x_test[0]

array([4.00e+01, 1.00e+00, 1.01e+02, 0.00e+00, 4.00e+01, 0.00e+00,
       2.26e+05, 8.00e-01, 1.41e+02, 0.00e+00, 0.00e+00, 1.87e+02])

In [22]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)


In [23]:
y_train= np.expand_dims(y_train, 1)
x_train = np.expand_dims(x_train, 1)

y_test = np.expand_dims(y_test, 1)
x_test = np.expand_dims(x_test, 1)


In [24]:
#RNN Model
model = Sequential()
model.add(LSTM(25, return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.fit(x_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
8/8 [==============================] - 10s 8ms/step - loss: 0.2977
Epoch 2/10
8/8 [==============================] - 0s 8ms/step - loss: 0.2766
Epoch 3/10
8/8 [==============================] - 0s 9ms/step - loss: 0.2549
Epoch 4/10
8/8 [==============================] - 0s 8ms/step - loss: 0.2271
Epoch 5/10
8/8 [==============================] - 0s 8ms/step - loss: 0.1976
Epoch 6/10
8/8 [==============================] - 0s 8ms/step - loss: 0.1658
Epoch 7/10
8/8 [==============================] - 0s 9ms/step - loss: 0.1458
Epoch 8/10
8/8 [==============================] - 0s 9ms/step - loss: 0.1367
Epoch 9/10
8/8 [==============================] - 0s 9ms/step - loss: 0.1305
Epoch 10/10
8/8 [==============================] - 0s 9ms/step - loss: 0.1239


In [25]:
ypred = model.predict(x_test)
ypred

array([[[-0.04346111]],

       [[ 0.3547458 ]],

       [[ 0.39871973]],

       [[ 0.1317579 ]],

       [[ 0.25529566]],

       [[ 0.2611276 ]],

       [[ 0.5891606 ]],

       [[ 0.11045839]],

       [[ 0.43813685]],

       [[ 0.39642036]],

       [[ 0.337909  ]],

       [[ 0.63795006]],

       [[ 0.27963886]],

       [[ 0.12471117]],

       [[ 0.595874  ]],

       [[ 0.6423406 ]],

       [[ 0.04267825]],

       [[ 0.4006386 ]],

       [[ 0.1777356 ]],

       [[ 0.10884784]],

       [[ 0.48325494]],

       [[ 0.4025491 ]],

       [[ 0.42800206]],

       [[ 0.30881274]],

       [[ 0.3849477 ]],

       [[ 0.26583555]],

       [[ 0.24423876]],

       [[ 0.08669087]],

       [[ 0.03054503]],

       [[ 0.17948595]],

       [[-0.0163341 ]],

       [[ 0.57434934]],

       [[ 0.21431309]],

       [[ 0.256334  ]],

       [[ 0.33682266]],

       [[ 0.47845122]],

       [[ 0.31523848]],

       [[ 0.38342372]],

       [[-0.04023997]],

       [[ 1.0657107 ]],



In [26]:
y_test

array([[0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0]], dtype=int64)